In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
from torch import nn
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import random
from captum.attr import IntegratedGradients
from captum.attr import IntegratedGradients, LayerConductance, NeuronConductance
import shap
import warnings
from sklearn.feature_selection import VarianceThreshold
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_excel(r"Endocrine_disrupting_toxicity.xlsx", index_col=0)
print(data.shape)
data.isnull().sum()
data.dropna(axis=1,inplace=True)
print(data.shape)
X = data.iloc[:, 1:]
Y = data.iloc[:, 0]
name = []
name = data.columns
name_X = []
name_X = X.columns
scaler = MinMaxScaler() 
scaler = scaler.fit(X)
X = scaler.transform(X)
X = pd.DataFrame(X)
X.columns = name_X

In [ ]:
Xtrain,Xtest,Ytrain,Ytest = train_test_split(X, Y, test_size=0.2, random_state=2346)
Xtrain = torch.tensor(Xtrain.values, dtype = torch.float32, requires_grad=True)
Ytrain = torch.tensor(Ytrain.values, dtype = torch.float32)
Ytrain = Ytrain.reshape(-1, 1)
Xtest = torch.tensor(Xtest.values, dtype = torch.float32, requires_grad=True)
Ytest = torch.tensor(Ytest.values, dtype = torch.float32)

def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

setup_seed(11)


class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.Tanh()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out


input_size = 198
hidden_size = 100
output_size = 1
model = Net(input_size, hidden_size, output_size)


criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.002)


num_epochs = 95
train_r2_list = []
test_r2_list = []
loss_list = []
for epoch in range(num_epochs):
    
    outputs = model(Xtrain)
    loss = criterion(outputs, Ytrain)

    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    
    train_r2 = r2_score(Ytrain.detach().numpy(), outputs.detach().numpy())
    test_r2 = r2_score(Ytest.detach().numpy(), model(Xtest).detach().numpy())
    train_r2_list.append(train_r2)
    test_r2_list.append(test_r2)
    
    loss_list.append(loss.item())


plt.plot(test_r2_list, label='Test R²')
plt.legend()
plt.savefig('FigureA.jpeg', dpi=800)
plt.show()

In [ ]:
from captum.attr import LayerConductance

lc = LayerConductance(model, model.fc1)

attr = lc.attribute(Xtest)
neuron_importance = attr.mean(dim=0).detach().numpy()

sorted_neurons = np.argsort(neuron_importance)[::-1]

for i, neuron_idx in enumerate(sorted_neurons):
    print(f"Neuron {neuron_idx+1}：{neuron_importance[neuron_idx]}")

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch

data = {
    'Neuron 1': -1.0436538457870483,
    'Neuron 2': -0.9643536806106567,
    'Neuron 3': -0.8725631833076477,
    'Neuron 4': -0.8477023243904114,
    'Neuron 5': -0.8362962007522583,
    'Neuron 6': 0,
    'Neuron 7': -0.8157883286476135,
    'Neuron 8': -0.8106172680854797,
    'Neuron 9': -0.7873838543891907,
    'Neuron 10': -0.7696040272712708,
    'Neuron 11': -0.7640208005905151,
    #'Neuron 11':2.4897756576538086,
    #'Neuron 12':1.9344542026519775,
    #'Neuron 13':1.3854787349700928,
    #'Neuron 14':1.3820840120315552,
    #'Neuron 15':1.1767284870147705,
    #'Neuron 16':1.1616559028625488,
    #'Neuron 17':1.1439871788024902,
    #'Neuron 18':0.7889372706413269,
    #'Neuron 19':0.7652159333229065,
    #'Neuron 20':0.6409977674484253,
}


'''
'Neuron 6': 6.597263813018799,
'Neuron 97':6.399002552032471,
'Neuron 18':5.87737512588501,
'Neuron 22':5.4173688888549805,
'Neuron 17':4.501695156097412,
'Neuron 87':4.278412818908691,
'Neuron 88':4.137702465057373,
'Neuron 65':3.1936779022216797,
'Neuron 29':3.1493241786956787,
'Neuron 14':2.881157875061035,
'Neuron 49':2.4897756576538086,
'Neuron 64':1.9344542026519775,
'Neuron 56':1.3854787349700928,
'Neuron 21':1.3820840120315552,
'Neuron 52':1.1767284870147705,
'Neuron 31':1.1616559028625488,
'Neuron 79':1.1439871788024902,
'Neuron 26':0.7889372706413269,
'Neuron 80':0.7652159333229065,
'Neuron 98':0.6409977674484253,
'''



features = [
    'qed',
    'EState_VSA2',
    'Chi0v',
    'LabuteASA',
    'ExactMolWt',
    'MolWt',
    'MolMR',
    'FpDensityMorgan3',
    'HeavyAtomCount',
    'MinEStateIndex',
    '   ',
    'MolLogP',
    'HeavyAtomMolWt',
    'PEOE_VSA6',
    'Chi1',
    'SMR_VSA5',
    'MinPartialCharge',
    'NumValenceElectrons',
    'EState_VSA1',
    'Chi1v',
    'fr_bicyclic'
]

fig, ax = plt.subplots()


plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 8


for i, feature in enumerate(features):
    y = (len(features) - i - 1) * 30
    ax.add_patch(plt.Circle((0, y), radius=15, fill=True, color='#A5BBF1'))
    ax.text(-0.5, y-4, feature, ha='center')


ax.text(0, len(features[:2]) * 30 / 2 + 279, '.\n.\n.', fontsize=15, ha='center', va='center', linespacing=0.1)

for i in range(0, 11, 1):
    y = (10 - i) * 60
    value = data[f'Neuron {i+1}']
    if i == 5:  
        color = 'white'
    else:
        color = plt.cm.cool((value))
    ax.add_patch(plt.Circle((150, y), radius=15, fill=True, color=color))
    
    
    for j in range(len(features)):
        y2 = (len(features) - j - 1) * 30
        arrow = FancyArrowPatch((15, y2), (135, y), arrowstyle='-', color='#7F7F7F')
        ax.add_patch(arrow)

ax.add_patch(plt.Circle((300, 300), radius=15, fill=True, color='#A5BBF1'))
ax.text(300, 297.5, 'Prediction', ha='center')

for i in range(11):
    y = (10 - i) * 60
    arrow = FancyArrowPatch((165, y), (285, 300), arrowstyle='-', color='#7F7F7F')
    ax.add_patch(arrow)
    
ax.text(150, len(features[:2]) * 30 / 2 + 279, '.\n.\n.', fontsize=15, ha='center', va='center', linespacing=0.1)

sm = plt.cm.ScalarMappable(cmap='cool', norm=plt.Normalize(-2, 1))
sm.set_array([])
cbar = fig.colorbar(sm)
cbar.set_label('Value')

ax.set_xlim(-30, 330)
ax.set_ylim(-30, len(features) * 30 + 30)
ax.axis('off')

plt.savefig('FigureC.jpeg', dpi=800, bbox_inches='tight')
plt.show()